In [ ]:
import pandas as pd
from string import Template
from pathlib import Path

import warnings
warnings.simplefilter("ignore")

import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

data_path = Path('/kaggle/input/kaggle-llm-science-exam')

## We'll use `FLAN-T5-base` from Kaggle's Model Hub

You'll probably want to turn on the GPU option for the notebook! (Remember though, since this is a Code competition, you'll need to set Internet to Off for Notebook submissions to the competition.)

In [ ]:
llm = '/kaggle/input/flan-t5/pytorch/base/2'


device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = T5ForConditionalGeneration.from_pretrained(llm).to(device)
tokenizer = T5Tokenizer.from_pretrained(llm)

In [ ]:
tokenizer

The data is formatted as follows. For each `prompt` (e.g., the question) there are five possible answers labeled `[A-E]`. Only one of the answers is correct.

In [ ]:
test = pd.read_csv(data_path / 'test.csv', index_col='id')
test.head()

## Creating a preamble template

How you format your prompt to input to the LLM can make a big difference in the output you get. Here, we try to instruct the LLM to rank all of the options from most likely to least likely.

In [ ]:
preamble = \
    'Answer the following question by outputting the letters A, B, C, D, and E '\
    'in order of the most likely to least likely to be correct option. The predictions must be among the option A, B , C, D and E only'

template = Template('$preamble\n\n$prompt\n\nA) $a\nB) $b\nC) $c\nD) $d\nE) $e')

In [ ]:
template

In [ ]:
test.loc[0, 'prompt']

In [ ]:
def format_input(df, idx):
    
    prompt = df.loc[idx, 'prompt']
    a = df.loc[idx, 'A']
    b = df.loc[idx, 'B']
    c = df.loc[idx, 'C']
    d = df.loc[idx, 'D']
    e = df.loc[idx, 'E']

    input_text = template.substitute(
        preamble=preamble, prompt=prompt, a=a, b=b, c=c, d=d, e=e)
    
    return input_text

This is an example of a formatted question that would be used as input to the LLM.

In [ ]:
print(format_input(test, 2))

In [ ]:
inputs = tokenizer(format_input(test, 0), return_tensors="pt").to(device)
outputs = model.generate(**inputs)
answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)

print(answer)

## Post-processing

You can see from the above that the LLM did not properly follow instructions. You'll need to figure out how to ensure your model provides at least the top three predictions, and have checks and post-processing in place for when they don't (such as in our example!)

This notebook provides a naive and **very** fragile example of how to do this. You'll want to make something more rubust!

In [ ]:
def post_process(predictions):
    valid = set(['A', 'B', 'C', 'D', 'E'])
    # If there are no valid choices, return something and hope for partial credit
    if set(predictions).isdisjoint(valid):
        final_pred = 'A B C D E'
    else:
        final_pred = []
        for prediction in predictions:
            if prediction in valid:
                final_pred += prediction
        # add remaining letters
        to_add = valid - set(final_pred)
        final_pred.extend(list(to_add))
        # put in space-delimited format
        final_pred = ' '.join(final_pred)
        
    return final_pred

## Making a submission

We can now make a simple script to make a submission to the competition.

In [ ]:
submission = pd.read_csv(
    data_path / 'sample_submission.csv', index_col='id')

for idx in test.index:
    inputs = tokenizer(format_input(test, idx), return_tensors="pt").to(device)
    outputs = model.generate(**inputs)
    answer = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    submission.loc[idx, 'prediction'] = post_process(answer)

You can include all five possible answers, but only the first three will be counted!

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv')